In [226]:
import h5py as h5
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import comlex_anal
from scipy.optimize import curve_fit
from scipy.special import lambertw


In [256]:
particlenumber=20
boxsize=8.0
D=1.0/6.0
R=1.0
alpha=1.0
lambda_plus=1.0
lambda_c=1.0
lambda_minus=1.0
tau=0.05
k11=[]
k_erban=[]
for lambda_plus in [1.0]:
    lambda_minus=1.0
    #lambda_plus=1.0
    lambda_c=1.0
    alpha=1.0
    length=2**14
    complexb=comlex_anal.Analy_Complex(D,R,lambda_plus,lambda_minus,lambda_c,alpha,length,boxsize,particlenumber,tau)
    #show_particle_alpha(D,R,lambda_plus,lambda_minus,lambda_c,alpha,length,boxsize,particlenumber,tau,k1_alpha_0)
#plt.legend(loc=3,fontsize='small')
#plt.savefig('./finalreport/data/chapman-limit-radial',dpi=300)
#plt.show()



    f = h5.File(complexb.path+"mean_values_particle.h5", "r")
    particle_number_c_mean=(f['norm_number_c'][:])
    particle_number_p_mean=(f['norm_number_p'][:])
    particle_number_s_mean=(f['norm_number_s'][:])
    t=(f['t'][:])
    best_vals2, covar2= curve_fit(p_t_modell,t,(particle_number_p_mean*complexb.particles)/(1.0*complexb.boxsize**3))
    best_vals1, covar1= curve_fit(s_t_modell,t,(particle_number_s_mean*complexb.particles)/(1.0*complexb.boxsize**3))
    best_vals, covar= curve_fit(es_t_modell,t,(particle_number_c_mean)/(1.0*complexb.boxsize**3))
    k11.append(best_vals2[0])
    k_erban1=4.0*np.pi*complexb.Diffusion*(complexb.reactiondistance-(np.sqrt(complexb.Diffusion/(complexb.micro_reactionrate_forward))*np.tanh(complexb.reactiondistance*np.sqrt((complexb.micro_reactionrate_forward)/complexb.Diffusion))))
    
    k_erban.append(4*np.pi*complexb.micro_reactionrate_forward/6)
    marker="+"
    linestyle='-'
    marker="+"
    linestyle='-'
    color="y"
    best_vals2, covar2= curve_fit(p_t_modell,t,(particle_number_p_mean*complexb.particles)/(1.0*complexb.boxsize**3))
    best_vals1, covar1= curve_fit(s_t_modell,t,(particle_number_s_mean*complexb.particles)/(1.0*complexb.boxsize**3))
    best_vals, covar= curve_fit(es_t_modell,t,(particle_number_c_mean)/(1.0*complexb.boxsize**3))
    k1=best_vals2[0]
    def s_t_modell_alpha(t,k1_alpha_0,kminus,kcomplex,h):
                    kplus=k1_alpha_0*t**(-h)
                    S_0=(complexb.particles*1.0)/(complexb.boxsize**3)
                    E_0=1.0/(complexb.boxsize**3)
            
                    S_t=((kminus+kcomplex)/kplus)*lambertw((S_0/((kminus+kcomplex)/kplus))*np.exp((-kcomplex*E_0*t+S_0)/((kminus+kcomplex)/kplus)))
                    return S_t.real

    def p_t_modell_alpha(t,k1_alpha_0,kminus,kcomplex,h):

                    kplus=k1_alpha_0*t**(-h)
                    S_0=(complexb.particles*1.0)/(complexb.boxsize**3)
                    E_0=1.0/(complexb.boxsize**3)
                    S_t=((kminus+kcomplex)/kplus)*lambertw((S_0/((kminus+kcomplex)/kplus))*np.exp((-kcomplex*E_0*t+S_0)/((kminus+kcomplex)/kplus)))
                    ES_t=((E_0*S_t.real)/(((kminus+kcomplex)/kplus)+S_t.real))*(1-np.exp(-(((kminus+kcomplex)/kplus)+S_t.real)*kplus*t))
                    P_t=S_0-S_t.real-ES_t
                    return P_t
    def es_t_modell_alpha(t,k1_alpha_0,kminus,kcomplex,h):
                    kplus=k1_alpha_0*t**(-h)
                    S_0=(complexb.particles*1.0)/(complexb.boxsize**3)
                    E_0=1.0/(complexb.boxsize**3)
                    S_t=((kminus+kcomplex)/kplus)*lambertw((S_0/((kminus+kcomplex)/kplus))*np.exp((-kcomplex*E_0*t+S_0)/((kminus+kcomplex)/kplus)))
                    ES_t=((E_0*S_t.real)/(((kminus+kcomplex)/kplus)+S_t.real))*(1-np.exp(-(((kminus+kcomplex)/kplus)+S_t.real)*kplus*t))
                    return ES_t
    def makemean(k1,step):
                        i=0
                        index_old=0
                        t_neu=[]
                        t_increment=((step+1)/2)
                        k1_neu=[]
                        while i<complexb.length/step:
                            index=index_old+step
                            k1_neu.append(k1)
                            t_neu.append(t_increment)
                            t_increment=t_increment+step
                            index_old=index
                            i=i+1
                        return np.array(t_neu), np.array(k1_neu)
    S_0=(complexb.particles*1.0)/(complexb.boxsize**3)
    E_0=1.0/(complexb.boxsize**3)
    t_index,k1neu=makemean(k1,3)
    tneu=t[t_index]
    h=0.178405098325
    
    k1=2.6*2.39305767541
    plt.loglog(t[1:]/complexb.timestep, particle_number_c_mean[1:],marker=marker,linestyle="",markevery=np.logspace(0,np.log10(len(range(2**14))),60).astype('int')[2:] ,label="complex %.2f" %complexb.alpha) #of complex
    plt.plot(t[1:]/complexb.timestep, particle_number_s_mean[1:],linestyle=linestyle , label="substrate %.2f" %complexb.alpha) #of substrate
    plt.plot(t[1:]/complexb.timestep, particle_number_p_mean[1:],linestyle=linestyle,color="g"  ,label=" product " ) # of product

    plt.plot(tneu[1:]/complexb.timestep,p_t_modell_alpha(tneu, k1,complexb.micro_reactionrate_backward,complexb.micro_reactionrate_complex,h)[1:]/S_0,color="g",linestyle="--" )
    plt.plot(tneu[1:]/complexb.timestep,s_t_modell_alpha(tneu,k1,complexb.micro_reactionrate_backward,complexb.micro_reactionrate_complex,h)[1:]/S_0,color="r",linestyle="--" )
    plt.plot(tneu[1:]/complexb.timestep,es_t_modell_alpha(tneu,k1,complexb.micro_reactionrate_backward,complexb.micro_reactionrate_complex,h)[1:]*complexb.boxsize**3,color=color,linestyle="--" )


    print best_vals[0],best_vals1[0],best_vals2[0]
    print complexb.micro_reactionrate_backward,complexb.micro_reactionrate_complex,complexb.micro_reactionrate_forward
'''
f.close()
plt.plot(t,particle_number_c)
plt.plot(t,particle_number_p)
plt.plot(t,particle_number_s)

plt.show()
'''
f.close
#plt.plot(k1)
plt.show()

2.55197487528 2.47486273105 2.47469015331
1.0 1.0 1.0


In [223]:
plt.plot([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,2.0,3.0],k11,label="Simulation")
#plt.plot([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,2.0,3.0],np.ones(12)*(4*np.pi/6),linestyle="--", label="Diffusion limit")
plt.ylabel('$k_{+}$', fontsize=12)
plt.xlabel('$\lambda_+$', fontsize=12)
plt.legend(loc=3,fontsize='small')
plt.savefig('./Abschlussarbeit/data/diffusion-limit-concentrations1_k1',dpi=300)

plt.show()

In [215]:
particlenumber=20
boxsize=8.0
D=1.0/6.0
R=1.0
alpha=1.0
lambda_plus=1.0
lambda_c=1.0
lambda_minus=1.0
tau=0.05
k1=[]
k_erban=[]

for lambda_plus in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
    lambda_minus=1.0
    #lambda_plus=1.0
    lambda_c=1.0
    alpha=1.0
    length=2**14
    complexb=comlex_anal.Analy_Complex(D,R,lambda_plus,lambda_minus,lambda_c,alpha,length,boxsize,particlenumber,tau)
    #show_particle_alpha(D,R,lambda_plus,lambda_minus,lambda_c,alpha,length,boxsize,particlenumber,tau,k1_alpha_0)
#plt.legend(loc=3,fontsize='small')
#plt.savefig('./finalreport/data/chapman-limit-radial',dpi=300)
#plt.show()



    f = h5.File(complexb.path+"mean_values_particle.h5", "r")
    particle_number_c_mean=(f['norm_number_c'][:])
    particle_number_p_mean=(f['norm_number_p'][:])
    particle_number_s_mean=(f['norm_number_s'][:])
    t=(f['t'][:])
    best_vals2, covar2= curve_fit(p_t_modell,t,(particle_number_p_mean*complexb.particles)/(1.0*complexb.boxsize**3))
    best_vals1, covar1= curve_fit(s_t_modell,t,(particle_number_s_mean*complexb.particles)/(1.0*complexb.boxsize**3))
    best_vals, covar= curve_fit(es_t_modell,t,(particle_number_c_mean)/(1.0*complexb.boxsize**3))
    k1.append(best_vals2[0])
    k_erban1=4.0*np.pi*complexb.Diffusion*(complexb.reactiondistance-(np.sqrt(complexb.Diffusion/(complexb.micro_reactionrate_forward))*np.tanh(complexb.reactiondistance*np.sqrt((complexb.micro_reactionrate_forward)/complexb.Diffusion))))
    
    #k_erban.append(4*np.pi*complexb.micro_reactionrate_forward/6)

    print best_vals[0],best_vals1[0],best_vals2[0]
'''
f.close()
plt.plot(t,particle_number_c)
plt.plot(t,particle_number_p)
plt.plot(t,particle_number_s)

plt.show()
'''
f.close


0.397599290469 0.389303060855 0.389284379817
0.762142407196 0.731577123135 0.731543821206
1.07225272608 1.03365368375 1.03356906837
1.36514498062 1.3150431294 1.31496561918
1.59679336052 1.54437407791 1.54423800531
1.85361872361 1.78854614694 1.78844577795
2.04239426434 1.98664599865 1.98653992041
2.26491275036 2.15924957592 2.15914826525
2.45576834815 2.32141683062 2.32130258552
2.55197487528 2.47486273105 2.47469015331


<bound method File.close of <HDF5 file "mean_values_particle.h5" (mode r)>>

In [225]:

#plt.plot([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],k1,label="$\lambda_-=1$")
plt.plot([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,2.0,3.0],k11,label="simulated $k_+$")

plt.plot([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,2.0,3.0],np.ones(12)*(4*np.pi/6),linestyle="--", label="Diffusion limit")
plt.ylabel('$k_{+}$', fontsize=12)
plt.xlabel('$\lambda_+$', fontsize=12)
plt.legend(loc=3,fontsize='small')
plt.savefig('./Abschlussarbeit/data/diffusion-limit-concentractionlambdaminus_k1',dpi=300)

plt.show()

In [218]:
marker="+"
linestyle='-'
plt.loglog(t[1:], particle_number_c_mean[1:],marker=marker,linestyle="",markevery=np.logspace(0,np.log10(len(range(2**14))),60).astype('int')[2:] ,label="complex %.2f" %complexb.alpha) #of complex
plt.plot(t[1:], particle_number_s_mean[1:],linestyle=linestyle , label="substrate %.2f" %complexb.alpha) #of substrate
plt.plot(t[1:], particle_number_p_mean[1:],linestyle=linestyle,color="g"  ,label=" product " ) # of product
'''
plt.plot(tneu,p_t_modell(tneu, self.k1_alpha_0,1.0,1.0)/S_0,color="g",linestyle="--" )
plt.plot(tneu,s_t_modell(tneu,self.k1_alpha_0,1.0,1.0)/S_0,color="r",linestyle="--" )
plt.plot(tneu,es_t_modell(tneu,self.k1_alpha_0,self.micro_reactionrate_backward,self.micro_reactionrate_complex)*self.boxsize**3,color=color,linestyle="--" )
'''
plt.show()

In [228]:
def s_t_modell(t,kplus,kminus,kcomplex):
                    S_0=(complexb.particles*1.0)/(complexb.boxsize**3)
                    E_0=1.0/(complexb.boxsize**3)
                    kcomplex=complexb.micro_reactionrate_complex
                    kminus=complexb.micro_reactionrate_backward
                    #t= np.linspace(0,self.length*self.timestep,10000)
                    S_t=((kminus+kcomplex)/kplus)*lambertw((S_0/((kminus+kcomplex)/kplus))*np.exp((-kcomplex*E_0*t+S_0)/((kminus+kcomplex)/kplus)))
                    #S_t=((kminus+kcomplex)/(kplus*(t*0.05)**(np.log(self.alpha)/np.pi)))*lambertw((S_0/((kminus+kcomplex)/(kplus*(t*0.05)**(np.log(self.alpha)/np.pi))))*np.exp((-kcomplex*E_0*t+S_0)/((kminus+kcomplex)/(kplus*(t*0.05)**(np.log(self.alpha)/np.pi)))))
                    return S_t.real
def es_t_modell(t,kplus,kminus,kcomplex):
                    #kminus=1
                    kcomplex=complexb.micro_reactionrate_complex
                    kminus=complexb.micro_reactionrate_backward
                    S_0=(complexb.particles*1.0)/(complexb.boxsize**3)
                    E_0=1.0/(complexb.boxsize**3)
                    #t= np.linspace(0,self.length*self.timestep,10000)
                    S_t=((kminus+kcomplex)/kplus)*lambertw((S_0/((kminus+kcomplex)/kplus))*np.exp((-kcomplex*E_0*t+S_0)/((kminus+kcomplex)/kplus)))
                    ES_t=((E_0*S_t.real)/(((kminus+kcomplex)/kplus)+S_t.real))*(1-np.exp(-(((kminus+kcomplex)/kplus)+S_t.real)*kplus*t))

                    return ES_t
def p_t_modell(t,kplus,kminus,kcomplex):
                    #kminus=1
                    kcomplex=complexb.micro_reactionrate_complex
                    kminus=complexb.micro_reactionrate_backward
                    S_0=(complexb.particles*1.0)/(complexb.boxsize**3)
                    E_0=1.0/(complexb.boxsize**3)
                    #t= np.linspace(0,self.length*self.timestep,10000)
                    S_t=((kminus+kcomplex)/kplus)*lambertw((S_0/((kminus+kcomplex)/kplus))*np.exp((-kcomplex*E_0*t+S_0)/((kminus+kcomplex)/kplus)))
                    ES_t=((E_0*S_t.real)/(((kminus+kcomplex)/kplus)+S_t.real))*(1-np.exp(-(((kminus+kcomplex)/kplus)+S_t.real)*kplus*t))
                    P_t=S_0-S_t.real-ES_t
                    return P_t
                

best_vals2, covar2= curve_fit(p_t_modell,t,(particle_number_p_mean*complexb.particles)/(1.0*complexb.boxsize**3))
best_vals1, covar1= curve_fit(s_t_modell,t,(particle_number_s_mean*complexb.particles)/(1.0*complexb.boxsize**3))
best_vals, covar= curve_fit(es_t_modell,t,(particle_number_c_mean*complexb.particles)/(1.0*complexb.boxsize**3))
k1=best_vals2[0]

def makemean(k1,step):
                    i=0
                    index_old=0
                    t_neu=[]
                    t_increment=((step+1)/2)
                    k1_neu=[]
                    while i<complexb.length/step:
                        index=index_old+step
                        k1_neu.append(k1)
                        t_neu.append(t_increment)
                        t_increment=t_increment+step
                        index_old=index
                        i=i+1
                    return np.array(t_neu), np.array(k1_neu)
S_0=(complexb.particles*1.0)/(complexb.boxsize**3)
E_0=1.0/(complexb.boxsize**3)
t_index,k1neu=makemean(k1,3)
tneu=t[t_index]

In [235]:
marker="+"
linestyle='-'
color="y"
plt.loglog(t[1:], particle_number_c_mean[1:],marker=marker,linestyle="",markevery=np.logspace(0,np.log10(len(range(2**14))),60).astype('int')[2:] ,label="complex %.2f" %complexb.alpha) #of complex
plt.plot(t[1:], particle_number_s_mean[1:],linestyle=linestyle , label="substrate %.2f" %complexb.alpha) #of substrate
plt.plot(t[1:], particle_number_p_mean[1:],linestyle=linestyle,color="g"  ,label=" product " ) # of product

plt.plot(tneu[1:],p_t_modell(tneu, k1,complexb.micro_reactionrate_backward,complexb.micro_reactionrate_complex)[1:]/S_0,color="g",linestyle="--" )
plt.plot(tneu[1:],s_t_modell(tneu,k1,complexb.micro_reactionrate_backward,complexb.micro_reactionrate_complex)[1:]/S_0,color="r",linestyle="--" )
plt.plot(tneu[1:],es_t_modell(tneu,k1,complexb.micro_reactionrate_backward,complexb.micro_reactionrate_complex)[1:]*complexb.boxsize**3,color=color,linestyle="--" )


In [236]:
#k_erban=4.0*np.pi*complexb.Diffusion*(complexb.reactiondistance-(np.sqrt(complexb.Diffusion/(complexb.micro_reactionrate_forward))*np.tanh(complexb.reactiondistance*np.sqrt((complexb.micro_reactionrate_forward)/complexb.Diffusion))))
#plt.plot(tneu,p_t_modell(tneu,k_erban,1.0,1.0)/S_0,color="g",linestyle="-" )
#plt.plot(tneu,s_t_modell(tneu,k_erban,1.0,1.0)/S_0,color="r",linestyle="-" )
#plt.plot(tneu,es_t_modell(tneu,k_erban,complexb.micro_reactionrate_backward,complexb.micro_reactionrate_complex)*complexb.boxsize**3,color=color,linestyle="-" )

In [237]:
plt.show()

In [196]:
0.36*(1+0.1)

0.396